In [1]:
# import necessary modules 
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import netcdf
import math as math
import pandas as pd
from matplotlib import gridspec
import datetime as dt 
import glob
import seaborn as sns
import matplotlib as mpl
import bottleneck as bn

In [2]:
def get_datetime(year, month, day, sec):
        
    Y = year
    M = month
    D = day
    
    hours = sec/3600.
    
    if (hours>=24)&(hours<48):
        D=day+1
        hours = (sec/3600.)-24
        
    if hours>=48:
        D=day+2
        hours = (sec/3600.)-48   
        
    if (M==12) & (D>31):
        D=D-31
        M = 1
        Y = year+1
        
    if (M==1) & (D>31):
        D=D-31
        M = 2
        
    if (M==2) & (D>28):
        D=D-28
        M = 3

    h = int(np.floor(hours))
    
    minutes = (hours-h)*60.
    m = int(np.floor(minutes))

    seconds = (minutes-m)*60.
    s = int(np.floor(seconds))
        
    datetime_fmt = dt.datetime(Y,M,D,h,m,s)
        
    return(datetime_fmt)


def get_datetime_GV(month, day, time):
    datetime = []
    for k in range(len(time)):
        D = day 
        M = month
        Y = 2018

        sec = time[k]

        hours = sec/3600.
        h = int(np.floor(hours))

        minutes = (hours-h)*60.
        m = int(np.floor(minutes))

        seconds = (minutes-m)*60.
        s = int(np.floor(seconds))

        if h>=24:
            h = h-24
            D = D+1
            if (M==1) & (D>=31):
                M = M+1
                D = 1
            if (M==2) & (D>=28):
                M = M+1
                D = 1
            if (M==3) & (D>=31):
                M = M+1
                D = 1

        datetime_fmt = dt.datetime(int(Y),int(M),int(D),h,m,s)
        datetime.append(datetime_fmt)
    return(datetime)



In [3]:

# identify filenames for all flights.
opth= '/Users/cmcclus/Documents/McCluskey/projects/SOAR/SOC/SOC_obs/' #Christina's Mac
fspec='SOCRATESrf'
filename = sorted(glob.glob(opth+fspec+'*.nc'))

fillvalue = -32767

# read first flight of data to set up dataframe 
k = 0
GV_filename = filename[k]
nc_GV = netcdf.netcdf_file(GV_filename, 'r')

# -------------------------------------------------------------------------
# TIME is a pain. 
# Time = seconds since X
GV_seconds = np.array(nc_GV.variables['Time'][:])

# FlightDate = date of flight (MM/DD/YYYY)
GV_month = int(nc_GV.FlightDate[0:2])
GV_day = int(nc_GV.FlightDate[3:5])

# convert to datetime format 
GV_datetime = get_datetime_GV(GV_month, GV_day, GV_seconds)
# -------------------------------------------------------------------------

# -------------------------------------------------------------------------
# Met data
# PSXC = Pressure (hPa)
GV_P = np.array(nc_GV.variables['PSXC'][:], dtype='f')
GV_P[GV_P == fillvalue]=np.nan

# LAT = Latitude (deg N)
GV_lat = np.array(nc_GV.variables['LAT'][:], dtype='f')
GV_lat[GV_lat == fillvalue]=np.nan

# LON = Longitude (deg E)
GV_lon = np.array(nc_GV.variables['LON'][:], dtype='f')
GV_lon[GV_lon == fillvalue]=np.nan

# RHUM = Relative Humidity (%)
GV_RH = np.array(nc_GV.variables['RHUM'][:], dtype='f')
GV_RH[GV_RH == fillvalue]=np.nan

# ATX = air temperature (deg C)
GV_T = np.array(nc_GV.variables['ATX'][:], dtype='f')
GV_T[GV_T == fillvalue]=np.nan

# GEOPTH = geopotential height (m)
GV_Z = np.array(nc_GV.variables['GEOPTH'][:], dtype='f')
GV_Z[GV_Z == fillvalue]=np.nan

# CVCWCC = Liquid water content (g/m2) from CDP
GV_LWC_CDP = np.array(nc_GV.variables['CVCWCC'][:], dtype='f')
GV_LWC_CDP[GV_LWC_CDP == fillvalue]=np.nan

# CVINLET = CVI inlet flag, 0=CVI, 1=Total
GV_CVINLET = np.array(nc_GV.variables['CVINLET'][:], dtype='f')
GV_CVINLET[GV_CVINLET == fillvalue]=np.nan

# CVTS = CVI Temperature Sample Line (deg C)
GV_CVTS = np.array(nc_GV.variables['CVTS'][:], dtype='f')
GV_CVTS[GV_CVTS == fillvalue]=np.nan

# PLWCC = Corrected PMS-King Liquid Water Content (g/m3)
GV_PLWCC = np.array(nc_GV.variables['PLWCC'][:], dtype='f')
GV_PLWCC[GV_PLWCC == fillvalue]=np.nan

# PLWC2DCA_RWOI = Fast 2DC Liquid Water Content, All Particles (g/m3)
GV_LWC2DC = np.array(nc_GV.variables['PLWC2DCA_RWOI'][:], dtype='f')
GV_LWC2DC[GV_LWC2DC == fillvalue]=np.nan

# WIC = Vertical Velocity (m/s)
GV_w = np.array(nc_GV.variables['WIC'][:], dtype='f')
GV_w[GV_w == fillvalue]=np.nan

# GEOPTH = geopotential height (m)
GV_Z = np.array(nc_GV.variables['GEOPTH'][:], dtype='f')
GV_Z[GV_Z == fillvalue]=np.nan

# CONCN = CN concentration (/cc)
GV_Ncn = np.array(nc_GV.variables['CONCN'][:], dtype='f')
GV_Ncn[GV_Ncn == fillvalue]=np.nan

# WSC = GPS-Corrected Horizontal Wind Speed (m/s)
GV_ws = np.array(nc_GV.variables['WSC'][:], dtype='f')
GV_ws[GV_ws == fillvalue]=np.nan

# -------------------------------------------------------------------------

data = {'datetime':GV_datetime, 
        'pressure':GV_P, 
        'lat':GV_lat, 
        'lon':GV_lon, 
        'relhum':GV_RH, 
        'temp':GV_T, 
        'cwc_cdp':GV_LWC_CDP, 
        'w':GV_w, 
        'Z':GV_Z, 
        'Ncn':GV_Ncn, 
        'ws':GV_ws,
        'CVINLET':GV_CVINLET,
        'CVTS':GV_CVTS,
        'PLWCC':GV_PLWCC,
        'LWC2DC':GV_LWC2DC}

gv_df = pd.DataFrame(data=data)

# ------------------------------------------------------------------------- 
# particle distribution measurements 
GV_UHSAS_n = nc_GV.variables['CUHSAS_LWII'][:,0,1:] # #/cc (per bin)
GV_UHSAS_size = nc_GV.variables['CUHSAS_LWII'].CellSizes # micrometer

GV_UHSAS_CVI_n = nc_GV.variables['CUHSAS_CVIU'][:,0,1:] # #/cc
GV_UHSAS_CVI_size = nc_GV.variables['CUHSAS_CVIU'].CellSizes # micrometer

GV_CDP_n = nc_GV.variables['CCDP_RWIO'][:,0,1:] # #/cc
GV_CDP_size = nc_GV.variables['CCDP_RWIO'].CellSizes #micrometer

for i in range(len(filename)-1):
    k = i+1
    
    GV_filename = filename[k]
    nc_GV = netcdf.netcdf_file(GV_filename, 'r')
    
    # ----------
    # TIME is a pain. 
    # Time = seconds since X
    GV_seconds = np.array(nc_GV.variables['Time'][:])

    # FlightDate = date of flight (MM/DD/YYYY)
    GV_month = int(nc_GV.FlightDate[0:2])
    GV_day = int(nc_GV.FlightDate[3:5])

    # convert to datetime format 
    GV_datetime = get_datetime_GV(GV_month, GV_day, GV_seconds)
    # ----------

    # PSXC = Pressure (hPa)
    GV_P = np.array(nc_GV.variables['PSXC'][:], dtype='f')
    GV_P[GV_P == fillvalue]=np.nan

    # LAT = Latitude (deg N)
    GV_lat = np.array(nc_GV.variables['LAT'][:], dtype='f')
    GV_lat[GV_lat == fillvalue]=np.nan

    # LON = Longitude (deg E)
    GV_lon = np.array(nc_GV.variables['LON'][:], dtype='f')
    GV_lon[GV_lon == fillvalue]=np.nan

    # RHUM = Relative Humidity (%)
    GV_RH = np.array(nc_GV.variables['RHUM'][:], dtype='f')
    GV_RH[GV_RH == fillvalue]=np.nan

    # ATX = air temperature (deg C)
    GV_T = np.array(nc_GV.variables['ATX'][:], dtype='f')
    GV_T[GV_T == fillvalue]=np.nan

    # GEOPTH = geopotential height (m)
    GV_Z = np.array(nc_GV.variables['GEOPTH'][:], dtype='f')
    GV_Z[GV_Z == fillvalue]=np.nan
    
    # CVCWCC = Liquid water content (g/m2) from CDP
    GV_LWC_CDP = np.array(nc_GV.variables['CVCWCC'][:], dtype='f')
    GV_LWC_CDP[GV_LWC_CDP == fillvalue]=np.nan

    # WIC = Vertical Velocity (m/s)
    GV_w = np.array(nc_GV.variables['WIC'][:], dtype='f')
    GV_w[GV_w == fillvalue]=np.nan

    # GEOPTH = geopotential height (m)
    GV_Z = np.array(nc_GV.variables['GEOPTH'][:], dtype='f')
    GV_Z[GV_Z == fillvalue]=np.nan

    # CONCN = CN concentration (/cc)
    GV_Ncn = np.array(nc_GV.variables['CONCN'][:], dtype='f')
    GV_Ncn[GV_Ncn == fillvalue]=np.nan
    
    # WSC = GPS-Corrected Horizontal Wind Speed (m/s)
    GV_ws = np.array(nc_GV.variables['WSC'][:], dtype='f')
    GV_ws[GV_ws == fillvalue]=np.nan
    
    # CVINLET = CVI inlet flag, 0=CVI, 1=Total
    GV_CVINLET = np.array(nc_GV.variables['CVINLET'][:], dtype='f')
    GV_CVINLET[GV_CVINLET == fillvalue]=np.nan
        
    # CVTS = CVI Temperature Sample Line (deg C)
    GV_CVTS = np.array(nc_GV.variables['CVTS'][:], dtype='f')
    GV_CVTS[GV_CVTS == fillvalue]=np.nan
        
    # PLWCC = CDP Water/Ice Content (g/m3)
    GV_PLWCC = np.array(nc_GV.variables['PLWCC'][:], dtype='f')
    GV_PLWCC[GV_PLWCC == fillvalue]=np.nan
    
    # PLWC2DCA_RWOI = Fast 2DC Liquid Water Content, All Particles (g/m3)
    GV_LWC2DC = np.array(nc_GV.variables['PLWC2DCA_RWOI'][:], dtype='f')
    GV_LWC2DC[GV_LWC2DC == fillvalue]=np.nan
    
    data = {'datetime':GV_datetime, 
            'pressure':GV_P, 
            'lat':GV_lat, 
            'lon':GV_lon, 
            'relhum':GV_RH, 
            'temp':GV_T,
            'cwc_cdp':GV_LWC_CDP, 
            'w':GV_w, 
            'Z':GV_Z, 
            'Ncn':GV_Ncn,
            'ws':GV_ws,
            'CVINLET':GV_CVINLET,
            'CVTS':GV_CVTS,
            'PLWCC':GV_PLWCC,
            'LWC2DC':GV_LWC2DC}
   
    dataframe = pd.DataFrame(data=data)
    
    gv_df=pd.concat([gv_df,dataframe], ignore_index=True)
    
    # ------------------------------------------------------------------------- 
    # particle distribution measurements 
    UHSAS_n = nc_GV.variables['CUHSAS_LWII'][:,0,1:] # #/cc (per bin)
    
    UHSAS_CVI_n = nc_GV.variables['CUHSAS_CVIU'][:,0,1:] # #/cc

    CDP_n = nc_GV.variables['CCDP_RWIO'][:,0,1:] # #/cc
    # -------------------------------------------------------------------------
    
    GV_UHSAS_n = np.concatenate((GV_UHSAS_n, UHSAS_n), axis =0)
    GV_UHSAS_CVI_n = np.concatenate((GV_UHSAS_CVI_n, UHSAS_CVI_n), axis =0)
    GV_CDP_n = np.concatenate((GV_CDP_n, CDP_n), axis =0)
    
GV_UHSAS_n[GV_UHSAS_n==fillvalue]=np.nan
GV_UHSAS_CVI_n[GV_UHSAS_CVI_n==fillvalue]=np.nan
GV_CDP_n[GV_CDP_n==fillvalue]=np.nan
    

/Users/cmcclus/anaconda3/lib/python3.7/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


In [4]:
## for size distribution measurements, calculate running mean 

def rollavg_bottlneck(a,n):
    return bn.move_mean(a, window=n,min_count = None)


n_bins = (np.shape(GV_UHSAS_n)[1])
GV_UHSAS_n_rm = np.zeros(np.shape(GV_UHSAS_n))

for i in range(n_bins):
    GV_UHSAS_n_rm[:,i] = rollavg_bottlneck(GV_UHSAS_n[:,i], 10)


    
n_bins = (np.shape(GV_UHSAS_CVI_n)[1])
GV_UHSAS_CVI_n_rm = np.zeros(np.shape(GV_UHSAS_CVI_n))

for i in range(n_bins):
    GV_UHSAS_CVI_n_rm[:,i] = rollavg_bottlneck(GV_UHSAS_CVI_n[:,i], 10)


In [5]:
# calculate UHSAS bin midpoints
nbins = 99
GV_UHSAS_bins = np.zeros([nbins])
GV_UHSAS_binwidth = np.zeros([nbins])

for i in range(nbins):
    GV_UHSAS_bins[i] = (GV_UHSAS_size[i+1]+GV_UHSAS_size[i])/2.
    GV_UHSAS_binwidth[i] = (np.log10(GV_UHSAS_size[i+1])-np.log10(GV_UHSAS_size[i]))
    
GV_UHSAS_dNdlogDp = GV_UHSAS_n[:,:]/GV_UHSAS_binwidth


#running mean (rm)
GV_UHSAS_dNdlogDp_rm = GV_UHSAS_n_rm[:,:]/GV_UHSAS_binwidth

In [6]:
## calculate running mean of UHSAS number concentrations

Dp_lower = 5
# Dp_upper = -4 # corresponds to 0.9 microns 
Dp_upper = -8# corresponds to 0.8 microns

print('wing-mounted UHSAS, lower size (microns)',GV_UHSAS_bins[Dp_lower])
print('wing-mounted UHSAS, upper size (microns)',GV_UHSAS_bins[Dp_upper])

GV_nUHSAS_trunc = np.sum(GV_UHSAS_n[:,Dp_lower:Dp_upper], axis =1)
gv_df['nUHSAS_trunc'] = GV_nUHSAS_trunc

GV_nUHSAS_trunc_rm = np.sum(GV_UHSAS_n_rm[:,Dp_lower:Dp_upper], axis =1)
gv_df['nUHSAS_trunc_rm'] = GV_nUHSAS_trunc_rm


wing-mounted UHSAS, lower size (microns) 0.07015499472618103
wing-mounted UHSAS, upper size (microns) 0.8081250190734863


In [7]:
gv_df.columns

Index(['datetime', 'pressure', 'lat', 'lon', 'relhum', 'temp', 'cwc_cdp', 'w',
       'Z', 'Ncn', 'ws', 'CVINLET', 'CVTS', 'PLWCC', 'LWC2DC', 'nUHSAS_trunc',
       'nUHSAS_trunc_rm'],
      dtype='object')

## EXPORT TIMESERIES DATA AS DATAFRAME

In [8]:
## export dataframes 
save_directory = '/Users/cmcclus/Documents/McCluskey/projects/SOAR/SOC/SOC_research_themes/methods/flightlev_CAM6/'
gv_df.to_csv(path_or_buf=save_directory+'SOCRATESgv_UHSASqc_M2023.csv', sep = ',')

In [10]:
# identify filenames for all flights.
opth= '/Users/cmcclus/Documents/McCluskey/projects/SOAR/SOC/SOC_obs/GNI/' #Christina's Mac
fspec='GNI_GV_all_SOCRATES'
filename = sorted(glob.glob(opth+fspec+'*.nc'))


# read first flight of data to set up dataframe 
GNI_filename = filename[0]
GNI_nc = netcdf.netcdf_file(GNI_filename, 'r')


data = {'startdatetime':np.array(GNI_nc.variables['starttime_gni'][:]),
        'enddatetime':np.array(GNI_nc.variables['endtime_gni'][:]),
        'Z':np.array(GNI_nc.variables['Z_gv'][:], dtype='f'),
        'lon':np.array(GNI_nc.variables['lon_gv'][:], dtype='f'),
        'lat':np.array(GNI_nc.variables['lat_gv'][:], dtype='f'),
        'P':np.array(GNI_nc.variables['p_gv'][:], dtype='f'),
        'RH':np.array(GNI_nc.variables['rh_gv'][:], dtype='f'), 
        'T':np.array(GNI_nc.variables['T_gv'][:], dtype='f'),
        'Ts':np.array(GNI_nc.variables['Ts_gv'][:], dtype='f'),
        'w':np.array(GNI_nc.variables['w_gv'][:], dtype='f'),
        'ws':np.array(GNI_nc.variables['ws_gv'][:], dtype='f'),
        'Tdiff':np.array(GNI_nc.variables['Tdiff_calc'][:], dtype='f'),
        'Ncn':np.array(GNI_nc.variables['Ncn_gv'][:], dtype='f'),
        'GF':np.array(GNI_nc.variables['GF_calc'][:], dtype='f'),
        'n1p4':np.array(GNI_nc.variables['n1p4_gni'][:], dtype='f'),
        'Msalt':np.array(GNI_nc.variables['Msalt_gni'][:], dtype='f'), 
        'CVINLET':np.array(GNI_nc.variables['CVINLET_gv'][:], dtype='f'), 
        'cwc_cdp':np.array(GNI_nc.variables['cwc_cdp_gv'][:], dtype='f')}
        
df_GNI = pd.DataFrame(data=data)

# GNI size distributions 
GNI_bins = np.array(GNI_nc.variables['GNIbins'][:], dtype='f')
GNI_dNdlogDp = np.array(GNI_nc.variables['sd_gni'][:], dtype='f')

In [11]:
df_GNI.columns

Index(['startdatetime', 'enddatetime', 'Z', 'lon', 'lat', 'P', 'RH', 'T', 'Ts',
       'w', 'ws', 'Tdiff', 'Ncn', 'GF', 'n1p4', 'Msalt', 'CVINLET', 'cwc_cdp'],
      dtype='object')